### FAKE NEWS DETECTION

##### importing the libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string
import nltk

In [2]:
from nltk.stem.porter import PorterStemmer

In [3]:
# nltk.download()

In [4]:
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [6]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [7]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [8]:
df_fake["class"] = 0
df_true["class"] = 1 #creating a class column this will serve as the label of the data

In [9]:
df_true.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [10]:
df_fake.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [11]:
df_fake.shape

(23481, 5)

In [12]:
df_true.shape

(21417, 5)

In [13]:
df_merged = pd.concat([df_fake,df_true], axis = 0) #merging both dataframe frames into one dataframe

In [14]:
df_merged.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [15]:
df_merged = df_merged.sample(frac = 1) #shuffling the data

In [16]:
df_merged.head(15)

,title,text,subject,date,class
6206,Trump taps NFL owner Woody Johnson as ambassad...,WASHINGTON (Reuters) - President-elect Donald ...,politicsNews,"January 19, 2017",1
19890,Catalonia asks Spain for dialogue as independe...,MADRID (Reuters) - Catalonian leaders on Frida...,worldnews,"September 15, 2017",1
20482,OBAMA’S LAWLESS AMERICA: CA Police Tell Violen...,"Barack Obama s legacy will be a divided, lawle...",left-news,"Jun 3, 2016",0
9093,McCain says he 'misspoke' in blaming Obama for...,WASHINGTON (Reuters) - U.S. Senator John McCai...,politicsNews,"June 16, 2016",1
21556,[Video] KANSAS DEM MAYOR BRAGS ABOUT REDUCTION...,What would a speech from a modern Democrat be ...,left-news,"Jul 16, 2015",0
6386,Hillary Clinton Is Apparently Eyeing Elizabet...,While Donald Trump is considering Arizona s fo...,News,"May 13, 2016",0
3104,U.S. attorney general unveils 12-city partners...,WASHINGTON (Reuters) - The U.S. Justice Depart...,politicsNews,"June 20, 2017",1
19590,"Russia becomes Iraq Kurds' top funder, quiet a...","LONDON (Reuters) - Russia, the only major powe...",worldnews,"September 20, 2017",1
15508,Papua New Guinea court rejects bid to restore ...,SYDNEY (Reuters) - Papua New Guinea s Supreme ...,worldnews,"November 7, 2017",1
12097,PLAYING POLITICS? HERE’S WHY A JUDGE Will Unse...,,politics,"Dec 20, 2016",0


In [17]:
df_merged.reset_index(inplace = True) #resettting the index of the dataset
df_merged.drop(["index"], axis = 1, inplace = True)#dropping the old index of the dataset

In [18]:
df_merged.head(15)

,title,text,subject,date,class
0,Trump taps NFL owner Woody Johnson as ambassad...,WASHINGTON (Reuters) - President-elect Donald ...,politicsNews,"January 19, 2017",1
1,Catalonia asks Spain for dialogue as independe...,MADRID (Reuters) - Catalonian leaders on Frida...,worldnews,"September 15, 2017",1
2,OBAMA’S LAWLESS AMERICA: CA Police Tell Violen...,"Barack Obama s legacy will be a divided, lawle...",left-news,"Jun 3, 2016",0
3,McCain says he 'misspoke' in blaming Obama for...,WASHINGTON (Reuters) - U.S. Senator John McCai...,politicsNews,"June 16, 2016",1
4,[Video] KANSAS DEM MAYOR BRAGS ABOUT REDUCTION...,What would a speech from a modern Democrat be ...,left-news,"Jul 16, 2015",0
5,Hillary Clinton Is Apparently Eyeing Elizabet...,While Donald Trump is considering Arizona s fo...,News,"May 13, 2016",0
6,U.S. attorney general unveils 12-city partners...,WASHINGTON (Reuters) - The U.S. Justice Depart...,politicsNews,"June 20, 2017",1
7,"Russia becomes Iraq Kurds' top funder, quiet a...","LONDON (Reuters) - Russia, the only major powe...",worldnews,"September 20, 2017",1
8,Papua New Guinea court rejects bid to restore ...,SYDNEY (Reuters) - Papua New Guinea s Supreme ...,worldnews,"November 7, 2017",1
9,PLAYING POLITICS? HERE’S WHY A JUDGE Will Unse...,,politics,"Dec 20, 2016",0


In [19]:
df_merged.shape

(44898, 5)

In [20]:
manual_test_df = df_merged.tail(20) #storing the last 20 rows in a new variable

In [21]:
manual_test_df.to_csv('manual_test_dataset.csv') #storing the dataset as a csv

In [22]:
df_merged.drop(df_merged.tail(20).index,inplace = True) #dropping the last 20 rows from our orignal dataset

In [23]:
df_merged.shape #checking the new shape of our dataset

(44878, 5)

In [24]:
df_merged['class'].value_counts()

0    23470
1    21408
Name: class, dtype: int64

In [25]:
df_copy = df_merged.copy(deep = True) #COPY OF DATASET WITHOUT TOKINIZATION OR STEMMING

##### Data Preprocessing

In [26]:
def wordopt(text):
    text = text.lower() #change text to lower  case
    text = re.sub('\[.*?\]', '', text) #remove all regex in the square box
    text = re.sub('https?://\S+|www\.\S+', '', text)#removes web addresses starting with https or www
    text = re.sub('\n', '', text) #removes all tab or new lines
    text = re.sub('[^a-zA-Z]',' ',text)#remove all strings that contains non letters
    return text

In [27]:
df_merged["text"] = df_merged["text"].apply(wordopt)

In [28]:
print(df_merged['text'])

0        washington  reuters    president elect donald ...
1        madrid  reuters    catalonian leaders on frida...
2        barack obama s legacy will be a divided  lawle...
3        washington  reuters    u s  senator john mccai...
4        what would a speech from a modern democrat be ...
                               ...                        
44873    washington  reuters    ohio governor john kasi...
44874    amman  reuters    the syrian army and iranian ...
44875    patrick henningsen   st century wiredespite re...
44876    sittwe  myanmar  reuters    around         dis...
44877    yangon  reuters    the united states temporari...
Name: text, Length: 44878, dtype: object


##### Tokenization: splitting sentence into words

In [29]:
#from nltk.tokenize import word_tokenize

In [30]:
#df_merged['text'] = df_merged['text'].apply(word_tokenize)

In [31]:
#print(df_merged['text'])

##### Stop Word Removal

In [32]:
#df_merged['text'] = df_merged['text'].apply(lambda x:[item for item in x if item not in STOPWORDS])

In [33]:
#print(df_merged['text'])

##### Stemming

In [34]:
#ps = PorterStemmer()

In [35]:
#df_merged['text'] = df_merged['text'].apply(lambda x: stemming(x))

In [36]:
#print(df_merged['text'])

In [37]:
#df_merged['text'] = df_merged['text'].astype(str)

In [38]:
#print(df_merged['text'])

#### DATA EXPLORATION/VISUALIZATION

In [39]:
#df_merged['class'].value_counts()

In [40]:
#sns.barplot( x = ['Fake_news','True_news'] , y = df_merged['class'].value_counts())

## MODEL BUILDING/SELECTION

In [41]:
#df_merged['text'] = df_merged['text'].astype(str)

In [42]:
x = df_merged["text"]
y = df_merged["class"]

In [43]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

###### Convert text to vector

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test) 

### Logistic Regression


In [46]:
#from sklearn.linear_model import LogisticRegression 

In [47]:
#regressor = LogisticRegression()
#regressor.fit(xv_train,y_train)

In [48]:
#prediction=regressor.predict(xv_test)

In [49]:
#regressor.score(xv_test, y_test)

In [50]:
#from sklearn import metrics
#score = metrics.accuracy_score(y_test, prediction)
#print("Accuracy: ", score)

In [51]:
#print(classification_report(y_test, prediction))

### Decision Tree Classification


In [52]:
from sklearn.tree import DecisionTreeClassifier

In [53]:
classifier = DecisionTreeClassifier()
classifier.fit(xv_train, y_train)

DecisionTreeClassifier()

In [54]:
pred_dt = classifier.predict(xv_test)

In [55]:
classifier.score(xv_test, y_test)

0.9952094474153298

In [56]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4747
           1       1.00      0.99      0.99      4229

    accuracy                           1.00      8976
   macro avg       1.00      1.00      1.00      8976
weighted avg       1.00      1.00      1.00      8976



### Gradient Boosting Classifier

In [57]:
#from sklearn.ensemble import GradientBoostingClassifier

In [58]:
#GBC = GradientBoostingClassifier(random_state=0)
#GBC.fit(xv_train, y_train)

In [59]:
#pred_gbc = GBC.predict(xv_test)

In [60]:
#GBC.score(xv_test, y_test)

In [61]:
#print(classification_report(y_test, pred_gbc))

### Random Forest Classifier

In [62]:
#from sklearn.ensemble import RandomForestClassifier

In [63]:
from joblib import dump, load

dump(classifier,'DecisionTree_model_saved.joblib')

['DecisionTree_model_saved.joblib']